<a href="https://colab.research.google.com/github/eikalise/image_segmentation/blob/main/Alise_Cell_Quantification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Required installations

In [32]:
!pip install ultralytics

## Mount to Google Drive

In [33]:
from google.colab import drive
#Make sure that you mount to the correct place in google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Load and run the model

First line of the output will give you how many phase 1s and phase 2s

You can look at the prediction photo by clicking on the 4 icon on the left panel (files) --> runs --> detect --> predict#


In [34]:
# Load the YOLO model
import cv2
import time
import numpy as np
from ultralytics import YOLO

import matplotlib.pyplot as plt


# the model (maybe called last.pt or best.pt) must be uploaded to your google drive account
# provide the correct path to the model
model = YOLO('/content/drive/MyDrive/weights/last.pt')

# provide the correct path to the photo
photo = cv2.imread('/content/drive/MyDrive/4.jpg') #this is a raw image of the fermentation photo at 50x dilution in RGB

# predict # of phase 1 and phase 2 for each photo
gray = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)
clahe = cv2.createCLAHE(clipLimit=16, tileGridSize=(16, 16))
processed_photo = clahe.apply(gray)
enhanced_3ch = cv2.merge([processed_photo, processed_photo, processed_photo])


predict = model.predict(enhanced_3ch, show=True, save=True, save_txt=True, show_boxes= True, show_conf=False, show_labels=False, max_det=350)



WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()


0: 384x640 48 phase 1s, 190 phase 2s, 368.1ms
Speed: 4.9ms preprocess, 368.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict20
1 label saved to runs/detect/predict20/labels


# Live Detection by connecting to Microsoft
This is unnecessary for now

In [ ]:
!pip install msal

import msal
import requests

# Define your Azure app credentials
client_id = 'YOUR_CLIENT_ID'
client_secret = 'YOUR_CLIENT_SECRET'
tenant_id = 'YOUR_TENANT_ID'
authority = f'https://login.microsoftonline.com/{tenant_id}'
scope = ['https://graph.microsoft.com/.default']

# Create a confidential client application
app = msal.ConfidentialClientApplication(client_id, authority=authority, client_credential=client_secret)

# Acquire a token
result = app.acquire_token_for_client(scopes=scope)

if 'access_token' in result:
    access_token = result['access_token']
    print("Access token acquired")
else:
    print("Error acquiring access token")
    print(result)

# Define the SharePoint file URL and site details
sharepoint_site = 'YOUR_SHAREPOINT_SITE'
sharepoint_file_path = 'YOUR_FILE_PATH'
sharepoint_drive = 'YOUR_DRIVE_ID'

# Create the API URL
url = f"https://graph.microsoft.com/v1.0/sites/{sharepoint_site}/drives/{sharepoint_drive}/root:/{sharepoint_file_path}:/content"

# Make the request to download the file
headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    with open('downloaded_image.jpg', 'wb') as f:
        f.write(response.content)
    print("Image downloaded successfully")
else:
    print("Failed to download image")
    print(response.json())



image 1/1 /content/drive/MyDrive/pos_4_fr_1_1.png: 640x640 (no detections), 331.7ms
Speed: 4.3ms preprocess, 331.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)


##Live Detection by connecting camera usb to computer
Doesn't work on goodle colab -- needs Jupyter notebook or other alternatives

In [ ]:
import cv2
import time
import numpy as np
from ultralytics import YOLO

def apply_clahe(image, clip_limit=16.0, tile_grid_size=(8, 8)):

    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    return clahe.apply(image)

def live_inference_with_yolo_and_clahe(model, clip_limit=2.0, tile_grid_size=(8, 8), conf_threshold=0.05):
    """
    Perform live inference using a YOLO model from a camera feed with CLAHE enhancement on every frame.

    Parameters:
    model (YOLO): The loaded YOLO model.
    clip_limit (float): Threshold for contrast limiting.
    tile_grid_size (tuple): Size of the grid for the histogram equalization.
    conf_threshold (float): Confidence threshold for predictions.
    """
    # Initialize the camera
    camera = cv2.VideoCapture(0)  # Use '0' for the default camera, replace with camera index if necessary

    if not camera.isOpened():
        print("Error: Could not open camera.")
        return

    while True:
        # Capture frame-by-frame
        ret, frame = camera.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply CLAHE
        enhanced_frame = apply_clahe(gray_frame, clip_limit=clip_limit, tile_grid_size=tile_grid_size)

        # Stack the grayscale image to create a 3-channel image
        enhanced_frame_3ch = cv2.merge([enhanced_frame, enhanced_frame, enhanced_frame])

        # Run YOLOv8 inference on the enhanced frame
        results = model.predict(enhanced_frame_3ch, conf=conf_threshold)

        # Draw bounding boxes and labels on the enhanced grayscale image
        if results:
            for result in results[0].boxes:
                # Get bounding box coordinates
                x1, y1, x2, y2 = map(int, result.xyxy[0])

                # Draw the bounding box
                cv2.rectangle(enhanced_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

                # Draw the label
                label = f"{int(result.cls.item())} {result.conf.item():.2f}"
                cv2.putText(enhanced_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Display the resulting frame
        cv2.imshow('YOLOv8 Live Predictions with CLAHE', enhanced_frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close windows
    camera.release()
    cv2.destroyAllWindows()

# Specify the model path
model_path = '/content/drive/MyDrive/last.pt'  # Replace with your model's path

# Load the YOLO model
model = YOLO(model_path)

# Perform live inference with YOLO and CLAHE enhancement
live_inference_with_yolo_and_clahe(model, clip_limit=16.0, tile_grid_size=(8, 8), conf_threshold=0.05)


Error: Could not open camera.
